In [27]:
# Import the required libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# Read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Logo and unique identifier
    html.Div([
        html.Img(src='assets/grazioso_logo.png', style={'height': '100px'}),
        html.H1('Module Six Assignment Liu, Kevin', style={'textAlign': 'center'})
    ], style={'textAlign': 'center'}),

    html.Hr(),
    
    # Dropdown filter
    html.Div([
        dcc.Dropdown(
            id='filter-dropdown',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Tracking'},
                {'label': 'Dogs', 'value': 'Dog'},
                {'label': 'Cats', 'value': 'Cat'}
            ],
            value='all',
            placeholder='Filter by Animal Type',
            style={'width': '50%'}
        )
    ], style={'margin-bottom': '20px'}),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Update table data based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-dropdown', 'value')]
)
def update_table(filter_value):
    if filter_value == 'all':
        filtered_df = df
    else:
        filtered_df = df[df['animal_type'] == filter_value]
    return filtered_df.to_dict('records')

# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update Graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.histogram(dff, x='breed', title='Distribution by Breed')
        )
    ]

# Update Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    if row_ids:
        row_id = row_ids[0]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, 
                   center=[dff.iloc[row_id]['location_lat'], dff.iloc[row_id]['location_long']], zoom=10, 
                   children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=[dff.iloc[row_id]['location_lat'], dff.iloc[row_id]['location_long']], children=[
                           dl.Tooltip(dff.iloc[row_id]['breed']),
                           dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row_id]['name'])])
                       ])
                   ])
        ]
    return []

app.run_server(debug=True)

Dash app running on http://127.0.0.1:27450/
